In [1]:
import tensorflow as tf
sess = tf.InteractiveSession()

In [2]:
import numpy as np

from tensorflow.contrib.rnn import core_rnn_cell
from tensorflow.contrib.seq2seq.python.ops import attention_wrapper
from tensorflow.contrib.seq2seq.python.ops import beam_search_decoder
from tensorflow.contrib.seq2seq.python.ops import beam_search_ops
from tensorflow.contrib.seq2seq.python.ops import decoder
from tensorflow.python.framework import constant_op
from tensorflow.python.framework import dtypes
from tensorflow.python.framework import ops
from tensorflow.python.layers import core as layers_core
from tensorflow.python.ops import array_ops
from tensorflow.python.ops import nn_ops
from tensorflow.python.ops import variables
from tensorflow.python.platform import test

In [3]:
batch_size = 2
beam_width = 3
vocab_size = 5
end_token = 0
length_penalty_weight = 0.6

In [5]:
dummy_cell_state = array_ops.zeros([batch_size, beam_width])
beam_state = beam_search_decoder.BeamSearchDecoderState(
    cell_state=dummy_cell_state,
    log_probs=nn_ops.log_softmax(
        array_ops.ones([batch_size, beam_width])),
    lengths=constant_op.constant(
        2, shape=[batch_size, beam_width], dtype=dtypes.int32),
    finished=array_ops.zeros(
        [batch_size, beam_width], dtype=dtypes.bool))

logits_ = np.full([batch_size, beam_width, vocab_size],
                  0.0001)

In [7]:
dummy_cell_state.eval()

array([[ 0.,  0.,  0.],
       [ 0.,  0.,  0.]], dtype=float32)

In [9]:
beam_state.log_probs.eval()

array([[-1.09861231, -1.09861231, -1.09861231],
       [-1.09861231, -1.09861231, -1.09861231]], dtype=float32)

In [10]:
beam_state.lengths.eval()

array([[2, 2, 2],
       [2, 2, 2]], dtype=int32)

In [11]:
beam_state.finished.eval()

array([[False, False, False],
       [False, False, False]], dtype=bool)

In [13]:
logits_

array([[[ 0.0001,  0.0001,  0.0001,  0.0001,  0.0001],
        [ 0.0001,  0.0001,  0.0001,  0.0001,  0.0001],
        [ 0.0001,  0.0001,  0.0001,  0.0001,  0.0001]],

       [[ 0.0001,  0.0001,  0.0001,  0.0001,  0.0001],
        [ 0.0001,  0.0001,  0.0001,  0.0001,  0.0001],
        [ 0.0001,  0.0001,  0.0001,  0.0001,  0.0001]]])

In [14]:
logits_[0, 0, 2] = 1.9
logits_[0, 0, 3] = 2.1
logits_[0, 1, 3] = 3.1
logits_[0, 1, 4] = 0.9
logits_[1, 0, 1] = 0.5
logits_[1, 1, 2] = 2.7
logits_[1, 2, 2] = 10.0
logits_[1, 2, 3] = 0.2
logits = ops.convert_to_tensor(logits_, dtype=dtypes.float32)
log_probs = nn_ops.log_softmax(logits)

In [16]:
logits_

array([[[  1.00000000e-04,   1.00000000e-04,   1.90000000e+00,
           2.10000000e+00,   1.00000000e-04],
        [  1.00000000e-04,   1.00000000e-04,   1.00000000e-04,
           3.10000000e+00,   9.00000000e-01],
        [  1.00000000e-04,   1.00000000e-04,   1.00000000e-04,
           1.00000000e-04,   1.00000000e-04]],

       [[  1.00000000e-04,   5.00000000e-01,   1.00000000e-04,
           1.00000000e-04,   1.00000000e-04],
        [  1.00000000e-04,   1.00000000e-04,   2.70000000e+00,
           1.00000000e-04,   1.00000000e-04],
        [  1.00000000e-04,   1.00000000e-04,   1.00000000e+01,
           2.00000000e-01,   1.00000000e-04]]])

In [17]:
log_probs.eval()

array([[[ -2.88203597e+00,  -2.88203597e+00,  -9.82135952e-01,
          -7.82136023e-01,  -2.88203597e+00],
        [ -3.31980968e+00,  -3.31980968e+00,  -3.31980968e+00,
          -2.19909757e-01,  -2.41990948e+00],
        [ -1.60943794e+00,  -1.60943794e+00,  -1.60943794e+00,
          -1.60943794e+00,  -1.60943794e+00]],

       [[ -1.73139989e+00,  -1.23149991e+00,  -1.73139989e+00,
          -1.73139989e+00,  -1.73139989e+00],
        [ -2.93801022e+00,  -2.93801022e+00,  -2.38110110e-01,
          -2.93801022e+00,  -2.93801022e+00],
        [ -1.00000916e+01,  -1.00000916e+01,  -1.91670173e-04,
          -9.80019188e+00,  -1.00000916e+01]]], dtype=float32)